In [1]:
import pandas as pd

In [28]:
df = pd.read_csv("fire_data.csv")

In [ ]:
df.head()

### 필요없는 컬럼 제거

In [29]:
columns_to_drop = [

    'prcs_result_se_nm',  # 처리결과구분명
    'hmg_ymd',            # 귀소일자
    'hmg_tm',             # 귀소시각
    'hmg_yr',             # 귀소연도
    'hmg_mnth',           # 귀소월
    'hmg_day',            # 귀소일
    'hmg_hour',           # 귀소시
    'hmg_min',            # 귀소분
    'sido_nm',            # 시도명
    'time_unit_msnf',     # 시간단위적설량
    'time_unit',          # 시간단위가시거리
    'acdnt_place_nm',     # 사고장소명
    'msfrtn_resc_reprt_no',  # 재난구조보고서번호
    'dsp_ymd',               # 출동일자
    'dsp_tm',                # 출동시각
    'dsp_yr',                # 출동연도
    'dsp_mnth',              # 출동월
    'dsp_day',               # 출동일
    'dsp_hour',              # 출동시
    'dsp_min',               # 출동분
    'resc_cmptn_tm',         # 구조완료시각
    'resc_cmptn_yr',         # 구조완료연도
    'resc_cmptn_mnth',       # 구조완료월
    'resc_cmptn_day',        # 구조완료일
    'cty_frmvl_se_nm',       # 도시농촌구분명
    'emd_se_nm'              # 읍면동구분명
]

df = df.drop(columns=columns_to_drop)


In [ ]:
df.info()

### 좌표 변환

In [32]:
import pandas as pd  # pandas 라이브러리 임포트
from pyproj import Proj, transform  # pyproj에서 Proj 및 transform 임포트


# TM5186 좌표계와 WGS 84 좌표계의 프로젝션 정의

# tm5186 = Proj(proj='tmerc', lat_0=38.5, lon_0=126.0, k=1, x_0=200000, y_0=500000, ellps='GRS80')

# 서울데이터
# tm5186 = Proj(proj='tmerc', lat_0=37.5665, lon_0=126.978, k=1, x_0=200000, y_0=500000, ellps='GRS80')


tm5186 = Proj(proj='tmerc', lat_0=37.5665, lon_0=126.978, k=1, x_0=200000, y_0=500000, ellps='GRS80')# TM5186 프로젝션 정의
wgs84 = Proj(proj='longlat', datum='WGS84')  # WGS 84 프로젝션 정의

# 변환 함수 정의ㄴ
def transform_coordinates(x, y):
    """주어진 TM5186 좌표를 WGS 84 좌표로 변환하는 함수"""
    lon, lat = transform(tm5186, wgs84, x, y)  # TM5186에서 WGS 84로 좌표 변환
    return lat, lon  # 변환된 위도와 경도를 반환

# 데이터프레임의 각 행에 대해 좌표 변환 적용
df['latitude'], df['longitude'] = zip(*df.apply(lambda row: transform_coordinates(row['gis_x_axis'], row['gis_y_axis']), axis=1))

# 결과 출력
print(df[['gis_x_axis', 'gis_y_axis', 'latitude', 'longitude']])  # 변환된 결과를 포함한 데이터프레임 출력


C:\Users\m\AppData\Local\Temp\ipykernel_8456\467047859.py:19: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat = transform(tm5186, wgs84, x, y)  # TM5186에서 WGS 84로 좌표 변환


       gis_x_axis   gis_y_axis   latitude   longitude
0     204810.2000  545291.8600  37.974551  127.032747
1     186122.4940  543977.2694  37.962614  126.820080
2     187477.7700  548528.3500  38.003636  126.835423
3     207280.7800  545612.4900  37.977423  127.060869
4     194072.3039  541646.1387  37.941699  126.910565
...           ...          ...        ...         ...
8383  202256.0414  550509.6913  38.021570  127.003693
8384  191057.2356  541494.4675  37.940308  126.876266
8385  211184.6700  548877.0400  38.006795  127.105353
8386  188521.9600  550298.6800  38.019599  126.847284
8387  210112.9159  541990.1776  37.944761  127.093052

[8388 rows x 4 columns]


In [37]:
import folium

#  126.043212  0.409606   125.875333  0.397716  

# 주어진 위도와 경도
latitude = 37.974551  
longitude = 127.032747

# Folium 지도 생성
m = folium.Map(location=[latitude, longitude], zoom_start=12)

# 위치에 작은 검정색 원 추가
folium.CircleMarker(location=[latitude, longitude], radius=8, color='black', fill=True, fill_opacity=0.6).add_to(m)

# 지도 출력 (주피터 노트북 사용 시)
m


### 지도 시각화 folium

In [ ]:
import folium

# 데이터의 첫 100개만 선택
df_subset = df.head(100)

m = folium.Map(location=[37.5665, 126.978], zoom_start=12)

# 상위 100개 좌표에 마커 추가
for idx, row in df_subset.iterrows():
    folium.Marker([row['Longitude'], row['Latitude']]).add_to(m)

# 지도 저장
m

In [45]:
import folium

# 기본 맵 생성: 서울의 대략적인 중앙 좌표로 설정
m = folium.Map(location=[37.5665, 126.978], zoom_start=12)

# 데이터 프레임 df의 좌표를 순회하며 마커 추가
for index, row in df.iterrows():
    folium.Marker(
        location=[row['gis_y_axis'], row['gis_x_axis']],
        popup=f"Index: {index}<br>X: {row['gis_x_axis']}<br>Y: {row['gis_y_axis']}",
        icon=folium.Icon(color='blue')
    ).add_to(m)

# 결과 맵을 HTML 파일로 저장
m.save("seoul_map.html")
